Building the AI Resume Screener with Python & Llama 3

Now, let’s get started with building the AI Resume Screener with Python and Llama3. We’ll break the process into three digestible parts: The Reader, The Brain, and The Execution.

Step 1: The Reader

In [15]:
# ===== READER =====

import fitz

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text.strip()

Step 2: The Brain

This is the key part of our AI Resume Screener. We aren’t just sending text; we are prompt engineering. We will give the AI a persona (of Senior Technical Recruiter) and specific constraints (JSON format):

In [17]:
# ===== RULE ENGINE =====

import re

REQUIRED_SKILLS = [
    "python",
    "pandas",
    "numpy",
    "scikit",
    "sql",
    "machine learning"
]

NICE_TO_HAVE = [
    "aws",
    "cloud",
    "nlp"
]


def rule_based_score(resume_text):
    resume_text_lower = resume_text.lower()

    score = 0
    found_skills = []
    missing_skills = []

    # Required skills weight = 10 each
    for skill in REQUIRED_SKILLS:
        if skill in resume_text_lower:
            score += 10
            found_skills.append(skill)
        else:
            missing_skills.append(skill)

    # Nice to have weight = 5 each
    for skill in NICE_TO_HAVE:
        if skill in resume_text_lower:
            score += 5
            found_skills.append(skill)

    final_score = min(score, 100)

    recommendation = "Interview" if final_score >= 60 else "Reject"

    return {
        "score": final_score,
        "found_skills": found_skills,
        "missing_skills": missing_skills,
        "recommendation": recommendation
    }

Step 3: The Execution

In [18]:
# ===== BRAIN (LIGHTWEIGHT LLM) =====

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True
)

model.to("cpu")


def generate_reasoning(resume_text, job_description, rule_result):

    prompt = f"""
    You are a technical recruiter.

    Job Description:
    {job_description}

    Resume Summary:
    {resume_text[:1500]}

    Rule-based score: {rule_result['score']}
    Found Skills: {rule_result['found_skills']}
    Missing Skills: {rule_result['missing_skills']}

    In 2 sentences, explain why the candidate should be {rule_result['recommendation']}.
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)

    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        do_sample=False,
        repetition_penalty=1.2
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [19]:
# ===== EXECUTION =====

job_description = """
We are looking for a Junior Data Scientist.
Must have:
- Python (Pandas, NumPy, Scikit-Learn)
- Experience with SQL
- Basic understanding of Machine Learning algorithms
- Good communication skills
Nice to have:
- Experience with AWS or Cloud deployment
- Knowledge of NLP
"""

pdf_path = "Ravi_Resume.pdf"

try:
    resume_text = extract_text_from_pdf(pdf_path)
    print("Resume loaded successfully.")
except Exception as e:
    print("Error loading resume:", e)
    raise SystemExit


print("Running Rule-Based ATS...")
rule_result = rule_based_score(resume_text)

print("Generating AI reasoning...\n")
reasoning = generate_reasoning(resume_text, job_description, rule_result)


print("----- HYBRID ATS REPORT -----")
print("Score:", rule_result["score"], "/100")
print("Recommendation:", rule_result["recommendation"])
print("Found Skills:", rule_result["found_skills"])
print("Missing Skills:", rule_result["missing_skills"])
print("AI Reasoning:", reasoning)

Resume loaded successfully.
Running Rule-Based ATS...
Generating AI reasoning...

----- HYBRID ATS REPORT -----
Score: 65 /100
Recommendation: Interview
Found Skills: ['python', 'pandas', 'numpy', 'scikit', 'sql', 'machine learning', 'nlp']
Missing Skills: []
AI Reasoning: Job Description: a Junior Data Scientist.
